In [1]:
import pandas as pd
import os
import sys
import time
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.ui import WebDriverWait
import re
import warnings
from datetime import datetime

warnings.filterwarnings(action='ignore')

driver = webdriver.Chrome(r"C:\Users\user\chromedriver.exe") # 자신의 크롬 드라이버 위치로 바꿔주세요

#검색어 조건에 따른 url 생성
def insta_seaching(word):
    url = "https://www.instagram.com/explore/tags/" + str(word)
    return url

#첫번째 게시물 클릭
def select_first(driver):
    first = driver.find_element(By.CSS_SELECTOR, "div._aagw").click()
    time.sleep(3)

#게시물 요소 수집
def get_content(driver):
    html = driver.page_source
    soup = BeautifulSoup(html, 'lxml')
    
    #본문 내용
    try:
        content = soup.select('div._a9zs')[0].text
        
    except:
        content = 'no content'
    
    #태그
    tags = re.findall(r'#[^\s#,\\]+', content)
    
    #날짜
    date = soup.select('time._aaqe')[0]['datetime'][:10]
    
    #좋아요
    try:
        like = soup.select_one('section._ae5m._ae5n._ae5o > div > div > span > a > span > span').text
        
    except Exception as e:
        print(e)
        like = 'no like'
     
    #위치
    try:
        place = soup.select('div._aaqm')[0].text
        
    except:
        place = 'no place'
        
    data = [content, date, like, place, tags]
    return data

#다음 게시물 클릭
def move_next(driver):
    right = driver.find_element(By.CSS_SELECTOR, "div._aaqg._aaqh")
    right.click()
    time.sleep(3)
    
options = webdriver.ChromeOptions()

driver.get('https://www.instagram.com')
time.sleep(3)

# ===============
#인스타 로그인
email = '2001.spaceyonissey' # 본인 인스타 아이디 입력
input_id = driver.find_element(By.NAME, "username")
input_id.clear()
input_id.send_keys(email)

password = 'zxcmnb12@' # 본인 인스타 비밀번호 입력
input_pw = driver.find_element(By.NAME, "password")
input_pw.clear()
input_pw.send_keys(password)
input_pw.submit()

time.sleep(5)

# ===============

#def insta_seaching(word):
#    url = "https://www.instagram.com/explore/tags/" + str(word)
#    return url

word = input("검색어를 입력하세요: ")
#word = str(word)
url = "https://www.instagram.com/explore/tags/" + str(word)

driver.get(url)
time.sleep(10)

# ===============

results = []

target = 20 #여기서 크롤링할 개수 입력

select_first(driver)

for i in range(target):
    time.sleep(5)
    
    try:
        data = get_content(driver)
        results.append(data)
        print(data)
        move_next(driver)
        
    except Exception as e:
        print(e)
        time.sleep(2)
        move_next(driver)

#csv파일로 저장
date = datetime.today().strftime('%Y-%m-%d')
results_df = pd.DataFrame(results)
results_df.columns = ['content', 'date', 'like', 'place', 'tags']
results_df.to_excel(date + '_about' +word+' insta crawling.xlsx')
results_df.to_csv('insta.csv')

driver.quit()

검색어를 입력하세요: 광고
['#광고⠀⠀⠀⠀⠀⠀세르지오 타키니 - 다마린도 컬러블록 반팔 폴로셔츠 셋업⠀⠀⠀⠀프리미엄 테니스 헤리티지 브랜드 ‘세르지오 타키니’⠀⠀⠀⠀⠀⠀고급스러운 컬러와 소재를 사용한 폴로셔츠!⠀⠀⠀⠀⠀⠀우븐 소재를 사용해 탄탄하고 쾌적하게 입기 좋은 여름 스포츠 필수템!⠀⠀⠀기장감은 살짝 짧아 다리가 길어보이는 효과까지!⠀⠀⠀⠀⠀⠀한 여름에도 땀 배출, 흡수가 잘 되는 기능성과 디자인까지 잡은⠀⠀⠀프리미엄 폴로셔츠 셋업!⠀⠀⠀⠀⠀⠀여름에 입기 컬러가 고급스러워 다양한 컬러 제품 전부 소장하고 싶네요!⠀⠀⠀⠀⠀⠀#sergiotacchini #세르지오타키니 #세르지오타키니크루⠀⠀⠀#tennis #premiumtennis #테니스 #테니스룩 #테니스복 #테니스웨어⠀⠀⠀#프리미엄테니스 #테니스셔츠 #폴로셔츠 #운동복⠀', '2023-07-09', '534', '', ['#광고⠀⠀⠀⠀⠀⠀세르지오', '#sergiotacchini', '#세르지오타키니', '#세르지오타키니크루⠀⠀⠀', '#tennis', '#premiumtennis', '#테니스', '#테니스룩', '#테니스복', '#테니스웨어⠀⠀⠀', '#프리미엄테니스', '#테니스셔츠', '#폴로셔츠', '#운동복⠀']]
['#광고 @discoveryexpedition_kr #iamadiscoverer #discoveryexpedition 안녕하세요 또스커버리입니다제 신발장에만 디스커버리 신발이 10켤레는 넘는 거 아시죠오,, 그만큼 디스커버리에 진심,,💚여행 쪽 일을 하게 되면서 무조건 발이 편한 신발을 선호하게 됐어요! 하루 2만보는 족히 걷게 되더라고요? 제 깐깐한 조건을 통과한 #캠퍼스플랫 을 소개합니다👟일단 신발 자체가 굉장히 가볍고요- 셀 구조로 된 바닥이 워킹 시 충격을 분산해줘서 많이 걸어도 피로함이 덜했어요🕊️ 접지력도 좋아서 트래킹화로도 손색이 없더라고요?!!여름이 찰떡처럼 어울리는 스니커즈💚 이번 주말도 저는 디스커버리와 함께 떠난답니다🕊️#디스커버리 #디스커버리운동화 #디스커

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"div._aaqg._aaqh"}
  (Session info: chrome=114.0.5735.199)
